In [ ]:
from jetbot import ObjectDetector
from jetbot import Camera
from jetbot import Robot
from jetbot import bgr8_to_jpeg
import torch
import torchvision
import torch.nn.functional as F
import cv2
import numpy as np
import time


# basic setup
model = ObjectDetector('ssd_mobilenet_v2_coco.engine')
camera = Camera.instance(width=300, height=300)
detections = model(camera.value)
print(detections)
robot = Robot()

In [ ]:
import ipywidgets.widgets as widgets
image_widget = widgets.Image(format='jpeg', width=300, height=300)

image_widget = widgets.Image(format='jpeg', width=300, height=300)
speed_widget = widgets.FloatSlider(value=0.4, min=0.0, max=1.0, description='speed')
left_alpha_widget = widgets.FloatSlider(value=0.7, min=0.0, max=2.0, description='left alpha')
right_alpha_widget = widgets.FloatSlider(value=0.8, min=0.0, max=2.0, description='right alpha')

display(widgets.VBox([
    image_widget,
    speed_widget,
    left_alpha_widget,
    right_alpha_widget
]))


width = int(image_widget.width)
height = int(image_widget.height)

target_obj_tall = 21
target_obj_short = 31

obstacle_obj_1 = 3
obstacel_obj_2 = 8

object_info_dict = {21:("tall_object", 200), 31:("short_object", 40), 3:("Apple", 30), 8:("Banana", 40)}

def detection_center(detection):
    """Computes the center x, y coordinates of the object"""
    bbox = detection['bbox']
    center_x = (bbox[0] + bbox[2]) / 2.0 - 0.5
    center_y = (bbox[1] + bbox[3]) / 2.0 - 0.5
    return (center_x, center_y)
    
def norm(vec):
    """Computes the length of the 2D vector"""
    return np.sqrt(vec[0]**2 + vec[1]**2)

def closest_detection(detections):
    """Finds the detection closest to the image center"""
    closest_detection = None
    for det in detections:
        center = detection_center(det)
        if closest_detection is None:
            closest_detection = det
        elif norm(detection_center(det)) < norm(detection_center(closest_detection)):
            closest_detection = det
    return closest_detection

def estimate_distance(detection):
    focal_length = 105.0  #need to caliborate your camera to get this
    obj_info = object_info_dict[detection['label']]
    bbox = detection['bbox']  
    obj_width = obj_info[1]
    obj_width_in_img = width * (bbox[2] - bbox[0])        
    dist = obj_width * focal_length / obj_width_in_img
    return dist


In [ ]:
is_local_steering_in_progress = False
is_moving_to_target_in_progress = False
prev_obstacle_label = -1

def local_steering(detections):
    # check if local steering is needed based on distance
    global is_local_steering_in_progress
    global prev_obstacle_label
    global is_moving_to_target_in_progress
    is_local_steering_needed = False
    
    obstacle = None
    obstacle_dist = 100000
    for det in detections:
        bbox = det['bbox']
        label = det['label']
       
        if(label == obstacle_obj_1 or label == obstacle_obj_2):
            dist = estimate_distance(det)
            is_local_steering_needed = dist < 80
            if(is_local_steering_needed):
                obstacle = det
                obstacle_dist = dist
            break
    
  

    if(is_local_steering_needed and obstacle is not None):  
        #if seen this before, skip local steering
        if(obstacle['label'] == prev_obstacle_label):
            return 
        is_local_steering_in_progress = True
        is_moving_to_target_in_progress = False
        print("start local steering")
        #local_steering code come here

        robot.left(speed=0.32)
        time.sleep(0.3)
        robot.forward(speed=0.4)
        time.sleep(0.8)

        robot.right(speed=0.32)
        time.sleep(0.3)
        robot.forward(speed=0.4)
        time.sleep(0.8)

        robot.right(speed=0.32)
        time.sleep(0.3)
        robot.forward(speed=0.4)
        time.sleep(0.8)
        
        robot.left(speed=0.32)
        time.sleep(0.3)
        robot.forward(speed=0.4)
        time.sleep(0.3)
        
        robot.stop()
        time.sleep(4.0)
        print("end local steering")
        prev_obstacle_label = obstacle['label']
        is_local_steering_in_progress = False
    
    
    
def is_target_reached(detections):
    # check if the car is close enough to the target
    for det in detections:
        bbox = det['bbox']
        label = det['label']
        if(label == target_obj_tall or label == target_obj_short):
            dist = estimate_distance(det)
            return dist < 20
            
    return False
   


def lock_and_move_to_target(detections):
    found_object = False
    for det in detections:
        bbox = det['bbox']
        label = det['label']
       
        if(label == target_obj_tall or label == target_obj_short):
            # move toward the target object
            center = detection_center(det)
            found_object = True
            robot.left_motor.alpha = left_alpha_widget.value
            robot.right_motor.alpha = right_alpha_widget.value
            robot.set_motors(
                float(speed_widget.value),
                float(speed_widget.value)
            )
            
            break
    if(found_object == False):
        robot.left(0.3)
            
    return

In [ ]:
def execute(change):
    
    image = change['new']
    
    # compute all detected objects
    detections = model(image)
    
    filtered_detections = []
    
    # visualized target and obstacle objects
    for det in detections[0]:
        bbox = det['bbox']
        label = det['label']
       
        if(label == target_obj_tall or label == target_obj_short):
            dist = estimate_distance(det)
            filtered_detections.append(det)
            cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (0, 0, 255), 2)
            cv2.putText(image, object_info_dict[label][0] + ":" + "{:.2f}".format(dist), (int(width * bbox[0]), int(height * bbox[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255),2)
        elif(label == obstacle_obj_1 or label == obstacle_obj_2):
            dist = estimate_distance(det)
            filtered_detections.append(det)
            cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (0, 255, 0), 2)
            cv2.putText(image, object_info_dict[label][0] + ":" + "{:.2f}".format(dist), (int(width * bbox[0]), int(height * bbox[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0),2)
        else:
            cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (255, 0, 0), 2)
            cv2.putText(image, str(label), (int(width * bbox[0]), int(height * bbox[1]) + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0),2)
      
         
    
    # update image widget
    image_widget.value = bgr8_to_jpeg(image)
    
    if(is_target_reached(filtered_detections)):
        camera.unobserve_all()
        robot.stop()
        return
    
    # if in local steering mode, return
    if(is_local_steering_in_progress):
        print("is_local_steering_in_progress")
        return
    
    
    if(is_local_steering_in_progress == False):
        lock_and_move_to_target(filtered_detections)
    

    
execute({'new': camera.value})

camera.unobserve_all()
camera.observe(execute, names='value')

In [ ]:
camera.unobserve_all()
time.sleep(1.0)
robot.stop()